In [ ]:
import sys
from pathlib import Path
from os import getcwd, remove
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent.parent))

In [ ]:
import pandas as pd
import numpy as np
from os.path import join

from datetime import datetime, date, timedelta
import random



from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace

from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 200)


exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

random.seed(10)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
tabela_base = pd.read_excel('ABRIR_HOSPITAIS.xlsx')

cols = ['UTI ADULTO','UTI  PEDIÁTRICA','ENF. ADULTO','ENF. PEDIÁTRICA','TOTAL']
for j in cols:
    tabela_base[j] = tabela_base[j].fillna(0).astype(int)

tabela_base['hospital_name'] = tabela_base['Município'] + tabela_base['Hospital']
tabela_base

In [ ]:
len(tabela_base['hospital_name'].unique())

In [ ]:
# DEFINIÇÕES
n_testes = 30000
testes_per_caixas = 20
n_caixinhas = n_testes / testes_per_caixas

filename = f"testes_hospitais.xlsx"

In [ ]:
tabela_base['CAIXAS'] = 1

In [ ]:
partial_total = tabela_base['CAIXAS'].sum()
partial_total

In [ ]:
testes_adicionais = random.choices(tabela_base['hospital_name'], weights=tabela_base['TOTAL'], k=int(n_caixinhas - partial_total))

In [ ]:
tabela_base = tabela_base.join(pd.Series(testes_adicionais, name='CAIXAS_ADICIONAIS').value_counts(), on='hospital_name')

In [ ]:
tabela_base['CAIXAS'] = tabela_base['CAIXAS'] + tabela_base['CAIXAS_ADICIONAIS'].fillna(0)
del(tabela_base['CAIXAS_ADICIONAIS'])
tabela_base

In [ ]:
# tabela_base['RS'] = tabela_base['RS'].astype(int)
# tabela_base['CAIXAS'] = tabela_base['CAIXAS'].astype(int)
tabela_base['TESTES'] = tabela_base['CAIXAS'] * testes_per_caixas
# tabela_base.loc['TOTAL', ['Populacao', 'CAIXAS', 'TESTES']] = tabela_base[['Populacao', 'CAIXAS', 'TESTES']].sum()
tabela_base

In [ ]:
tabela_base = tabela_base.set_index(['Macrorregião', 'Município']).drop(columns={'CAIXAS', 'hospital_name'})

In [ ]:
tabela_base

In [ ]:
writer = pd.ExcelWriter(join(default_output,'testes', filename),
                    engine='xlsxwriter')

In [ ]:
tabela_base.to_excel(writer,'hospitais')

In [ ]:
writer.save()
writer.close()